# Static ROM

This example shows how to create a static reduced order model (ROM)
in Twin Builder and run a transient simulation.

> **Note:** This example uses functionality only available in Twin Builder 2024 R2 and later.

Keywords: **Twin Builder**, **Static ROM**.

## Perform imports and define constants

Perform required imports.

In [1]:
import os
import shutil
import tempfile
import time

from ansys.aedt.core import TwinBuilder
from ansys.aedt.core.examples import downloads

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Set up input data

The following files are downloaded along with the
other project data used to run this example.

In [4]:
source_snapshot_data_zipfilename = "Ex1_Fluent_StaticRom.zip"
source_build_conf_file = "SROMbuild.conf"
source_props_conf_file = "SROM_props.conf"

## Download example data

The following cell downloads the required files needed to run this example and
extracts them in a local folder named ``"Ex04"``.

In [5]:
downloads.download_twin_builder_data(
    file_name=source_snapshot_data_zipfilename,
    force_download=True,
    local_path=temp_folder.name,
)

downloads.download_twin_builder_data(source_build_conf_file, True, temp_folder.name)
downloads.download_twin_builder_data(
    source_props_conf_file, True, temp_folder.name
)

# Target folder to extract project files.
twin_builder_data_folder = os.path.join(temp_folder.name, "twin_builder")
data_folder = os.path.join(twin_builder_data_folder, "Ex04")

# Unzip training data and config file
downloads.unzip(
    os.path.join(twin_builder_data_folder, source_snapshot_data_zipfilename), data_folder
)
shutil.copyfile(
    os.path.join(twin_builder_data_folder, source_build_conf_file),
    os.path.join(data_folder, source_build_conf_file),
)
shutil.copyfile(
    os.path.join(twin_builder_data_folder, source_props_conf_file),
    os.path.join(data_folder, source_props_conf_file),
)

C:\Users\ansys\AppData\Local\Temp\tmp8_pgqdfs.ansys\twin_builder\Ex04


'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmp8_pgqdfs.ansys\\twin_builder\\Ex04\\SROM_props.conf'

## Launch Twin Builder and build ROM component

Launch Twin Builder using an implicit declaration and add a new design with
the default setup for building the static ROM component.

In [6]:
project_name = os.path.join(temp_folder.name, "static_rom.aedt")
tb = TwinBuilder(
    project=project_name,
    version=AEDT_VERSION,
    non_graphical=NG_MODE,
    new_desktop=True,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_4418cb53-42d1-4d91-bd40-9504440c82fc.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project static_rom has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'Twin Builder_34B' of type Twin Builder.


PyAEDT INFO: Aedt Objects correctly read


## Configure AEDT

> **Note:** Only run the following cell if AEDT is not configured to run Twin Builder.
>
> The following cell configures AEDT and the schematic editor
> to use the ``Twin Builder`` configuration.
> The Static ROM feature is only available with a Twin Builder license.
> A cell at the end of this example restores the AEDT configuration. If your
> environment is set up to use the ``Twin Builder`` configuration, you do not
> need to run these sections.

In [7]:
current_desktop_config = tb._odesktop.GetDesktopConfiguration()
current_schematic_environment = tb._odesktop.GetSchematicEnvironment()
tb._odesktop.SetDesktopConfiguration("Twin Builder")
tb._odesktop.SetSchematicEnvironment(1)

In [8]:
# Get the static ROM builder object.
rom_manager = tb._odesign.GetROMManager()
static_rom_builder = rom_manager.GetStaticROMBuilder()

# Build the static ROM with the specified configuration file
confpath = os.path.join(data_folder, source_build_conf_file)
static_rom_builder.Build(confpath.replace("\\", "/"))

# Test if the ROM was created successfully.
static_rom_path = os.path.join(data_folder, "StaticRom.rom")
if os.path.exists(static_rom_path):
    tb.logger.info("Built intermediate rom file successfully at: %s", static_rom_path)
else:
    tb.logger.error("Intermediate rom file not found at: %s", static_rom_path)

# Create the ROM component definition in Twin Builder.
rom_manager.CreateROMComponent(static_rom_path.replace("\\", "/"), "staticrom")

PyAEDT INFO: Built intermediate rom file successfully at: C:\Users\ansys\AppData\Local\Temp\tmp8_pgqdfs.ansys\twin_builder\Ex04\StaticRom.rom


## Create schematic

Place components to create the schematic.

In [9]:
# Define the grid distance for ease in calculations.
G = 0.00254

# Place a dynamic ROM component.
rom1 = tb.modeler.schematic.create_component("ROM1", "", "staticrom", [40 * G, 25 * G])

# Place two excitation sources.
source1 = tb.modeler.schematic.create_periodic_waveform_source(
    None, "SINE", 2.5, 0.01, 0, 7.5, 0, [20 * G, 29 * G]
)
source2 = tb.modeler.schematic.create_periodic_waveform_source(
    None, "SINE", 50, 0.02, 0, 450, 0, [20 * G, 25 * G]
)

PyAEDT INFO: ModelerCircuit class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: ModelerTwinBuilder class has been initialized!


Connect components with wires.

In [10]:
tb.modeler.schematic.create_wire([[22 * G, 29 * G], [33 * G, 29 * G]])
tb.modeler.schematic.create_wire(
    [[22 * G, 25 * G], [30 * G, 25 * G], [30 * G, 28 * G], [33 * G, 28 * G]]
)

Class: ansys.aedt.core.modeler.circuits.object_3d_circuit.Wire

Enable storage of views.

In [11]:
rom1.set_property("store_snapshots", 1)
rom1.set_property("view1_storage_period", "10s")
rom1.set_property("view2_storage_period", "10s")

True

In [12]:
# Zoom to fit the schematic
tb.modeler.zoom_to_fit()

## Parametrize transient setup

Parametrize the default transient setup by setting the end time.

In [13]:
tb.set_end_time("300s")
tb.set_hmin("1s")
tb.set_hmax("1s")

True

## Solve transient setup

Solve the transient setup. Skipping this step in case the documentation is being built.

> **Note:** The following code can be uncommented.

tb.analyze_setup("TR")

## Get report data and plot using Matplotlib

Get report data and plot it using Matplotlib. The following code gets and plots
the values for the voltage on the pulse voltage source and the values for the
output of the dynamic ROM.

> **Note:** The following code can be uncommented, but it depends on the previous commented code.

e_value = "ROM1.outfield_mode_1"
x = tb.post.get_solution_data(e_value, "TR", "Time")
x.plot()
e_value = "ROM1.outfield_mode_2"
x = tb.post.get_solution_data(e_value, "TR", "Time")
x.plot()
e_value = "SINE1.VAL"
x = tb.post.get_solution_data(e_value, "TR", "Time")
x.plot()
e_value = "SINE2.VAL"
x = tb.post.get_solution_data(e_value, "TR", "Time")
x.plot()

## Close Twin Builder

After the simulation is completed, either close Twin Builder or release it.
All methods provide for saving the project before closing.

In [14]:
# Clean up the downloaded data.
shutil.rmtree(twin_builder_data_folder)

In [15]:
# Restore earlier desktop configuration and schematic environment.
tb._odesktop.SetDesktopConfiguration(current_desktop_config)
tb._odesktop.SetSchematicEnvironment(current_schematic_environment)

## Release AEDT

Release AEDT and close the example.

In [16]:
tb.save_project()
tb.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project static_rom Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``. If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell removes all temporary files, including the project folder.

In [17]:
temp_folder.cleanup()